In [60]:
import pandas as pd

pd_ds = pd.read_stata('./ZA7886_v1-0-0.dta')#.astype(str)
pd_ds = pd_ds.drop(columns=[col for col in pd_ds.columns if col not in ['isocntry', 'd17', 'sd18a', 'd72_2', 'd1', 'd11', 'netuse']])
# Replace 'Not mentioned' with -1, "Yes" with 1, "No" with 0
# pd_ds = pd_ds.replace({'Not mentioned': -1, 'Yes': 1, 'No': 0})
#ds = datasets.Dataset.from_pandas(pd_ds)
print(f"Total number of countries: {len(pd_ds['isocntry'].unique().tolist())}")

Total number of countries: 28


In [61]:
import torch
import torch.nn as nn

#pd_ds = pd_ds[:100]

# Transform the dataset into a format that can be used by the model
pd_ds = pd_ds[~pd_ds.isin(['DK (SPONT.)']).any(axis=1)]
# Remove the categories 'Refusal/Implausible', '15 years', '98 years' from the column 'd11'
pd_ds = pd_ds[~pd_ds['d11'].isin(['Refusal/Implausible', '15 years', '98 years'])]
# Remove the category 'Refusal (SPONT.)' from the column 'd1'
pd_ds = pd_ds[~pd_ds['d1'].isin(['Refusal (SPONT.)'])]

with pd.option_context('display.max_colwidth', None, 'display.max_rows', 3, 'display.max_columns', None):
    display(pd_ds.head(5))
    for col in pd_ds.columns:
        #display(pd_ds.value_counts(col))
        # Display the string of the category and the corresponding code in a table
        display(pd_ds[col].astype('category').cat.categories)
        original_series = pd_ds[col]
        converted_series = pd_ds[col].astype('category').cat.codes
        # Display the original series and the converted series side by side
        display(pd.concat([original_series, converted_series], axis=1))

,isocntry,d11,d1,netuse,sd18a,d72_2,d17
0,AT,57,2,Everyday/Almost everyday,Fairly satisfied,Tend to agree,Fairly optimistic
...,...,...,...,...,...,...,...
5,AT,19,7,Everyday/Almost everyday,Fairly satisfied,Tend to agree,Fairly optimistic


Index(['AT', 'BE', 'BG', 'CY', 'CZ', 'DE-E', 'DE-W', 'DK', 'EE', 'ES', 'FI',
       'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL',
       'PT', 'RO', 'SE', 'SI', 'SK'],
      dtype='object')

,isocntry,0
0,AT,0
...,...,...
26501,SK,27


Index(['Refusal/Implausible',            '15 years',                    16,
                          17,                    18,                    19,
                          20,                    21,                    22,
                          23,                    24,                    25,
                          26,                    27,                    28,
                          29,                    30,                    31,
                          32,                    33,                    34,
                          35,                    36,                    37,
                          38,                    39,                    40,
                          41,                    42,                    43,
                          44,                    45,                    46,
                          47,                    48,                    49,
                          50,                    51,                    52,
            

,d11,0
0,57,43
...,...,...
26501,38,24


...

,sd18a,0
0,Fairly satisfied,1
...,...,...
26501,Not very satisfied,2


Index(['Totally agree', 'Tend to agree', 'Tend to disagree',
       'Totally disagree', 'DK (SPONT.)'],
      dtype='object')

,d72_2,0
0,Tend to agree,1
...,...,...
26501,Tend to agree,1


Index(['Very optimistic', 'Fairly optimistic', 'Fairly pessimistic',
       'Very pessimistic', 'DK (SPONT.)'],
      dtype='object')

,d17,0
0,Fairly optimistic,1
...,...,...
26501,Fairly optimistic,1


In [62]:
# Convert the dataframe column 'isocntry' into a one-hot tensor. First, convert the string values into integers, and then use the one_hot function from torch.nn.functional
pd_ds['isocntry'] = pd_ds['isocntry'].astype('category').cat.codes
pd_ds['isocntry'] = pd_ds['isocntry'].map(lambda x: torch.nn.functional.one_hot(torch.tensor(x), num_classes=28))

# Replace the string values with their corresponding codes
for col in ['netuse', 'd17', 'sd18a', 'd72_2', 'd1', 'd11']:
    pd_ds[col] = pd_ds[col].astype('category').cat.codes

with pd.option_context('display.max_colwidth', None, 'display.max_rows', None, 'display.max_columns', None):
    display(pd_ds.head(1))

,isocntry,d11,d1,netuse,sd18a,d72_2,d17
0,"[tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)]",43,1,0,1,1,1


In [63]:
# Perform normalization of the columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
pd_ds['d11'] = scaler.fit_transform(pd_ds['d11'].values.reshape(-1, 1))
pd_ds['d1'] = scaler.fit_transform(pd_ds['d1'].values.reshape(-1, 1))
pd_ds['netuse'] = scaler.fit_transform(pd_ds['netuse'].values.reshape(-1, 1))
pd_ds['sd18a'] = scaler.fit_transform(pd_ds['sd18a'].values.reshape(-1, 1))
pd_ds['d72_2'] = scaler.fit_transform(pd_ds['d72_2'].values.reshape(-1, 1))
pd_ds['d17'] = scaler.fit_transform(pd_ds['d17'].values.reshape(-1, 1))

with pd.option_context('display.max_colwidth', None, 'display.max_rows', None, 'display.max_columns', None):
    display(pd_ds.head(5))

,isocntry,d11,d1,netuse,sd18a,d72_2,d17
0,"[tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)]",0.506173,0.111111,0.000000,0.333333,0.333333,0.333333
1,"[tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)]",0.160494,1.000000,0.000000,0.333333,0.333333,0.666667
2,"[tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)]",0.259259,1.000000,0.000000,0.000000,0.000000,0.333333
4,"[tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)]",0.580247,0.111111,0.333333,0.666667,0.000000,0.666667
5,"[tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)]",0.037037,0.666667,0.000000,0.333333,0.333333,0.333333


In [68]:
# Create a NN with 2 hidden layers of 50 neurons each
# The model outputs a one-hot vector of size 28
# It takes as input a vector of 6 features

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 28)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x
    
net = Net()

In [69]:
# Generate train and test splits
from sklearn.model_selection import train_test_split
train, test = train_test_split(pd_ds, test_size=0.2)

In [76]:
# Train the net
import torch.optim as optim
torch.manual_seed(0)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(len(train)):
        # get the inputs; data is a list of [inputs, labels]
        inputs = train.iloc[i, 1:].tolist()
        inputs = torch.tensor(inputs, dtype=torch.float)
        labels = train.iloc[i, 0]
        labels = torch.argmax(labels, dim=0)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
    # Evaluate
    correct = 0
    total = 0
    with torch.no_grad():
        for i in range(len(test)):
            inputs = test.iloc[i, 1:].tolist()
            inputs = torch.tensor(inputs, dtype=torch.float)
            labels = test.iloc[i, 0]
            labels = torch.argmax(labels, dim=0)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 0)
            total += 1
            correct += (predicted == labels)

print('Finished Training')

[1,  2000] loss: 3.260
[1,  4000] loss: 3.265
[1,  6000] loss: 3.261
[1,  8000] loss: 3.261
[1, 10000] loss: 3.257
[1, 12000] loss: 3.250
[1, 14000] loss: 3.244


In [66]:
# Test the net
dataiter = iter(pd_ds)

AttributeError: 'map' object has no attribute 'next'

In [ ]:
# Export the model to a torchscript model
traced_script_module = torch.jit.trace(net, inputs)
traced_script_module.save("model.pt")